In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division, unicode_literals

In [3]:
# __import__('datetime', fromlist=[u'datetime'])

In [4]:
import os 
from pprint import pprint

In [11]:
import numpy as np
import decimal

In [5]:
import streamm

In this getting started example we will calculate the electronic properties of methane with NWChem and the structural properties with LAMMPS

In [24]:
ME = streamm.Buildingblock('methane')

Create .xyz file using a molecular viewer, such as Avogadro (https://avogadro.cc/) or explicitely as in the structure.ipynb example.

In [25]:
ME.read_xyz()

In [35]:
print(ME.write_xyz_str())

 5 
 methane 
     C       0.00000000       0.00000000       0.00000000 
     H       0.69282032       0.69282032       0.69282032 
     H      -0.69282032      -0.69282032       0.69282032 
     H      -0.69282032       0.69282032      -0.69282032 
     H       0.69282032      -0.69282032      -0.69282032 



Looks good

Now let's create project and resource to keep track of our work

In [33]:
proj_i = streamm.Project('Methane_example')
res_i = streamm.Resource('local')

Add relative location of templates directory 

In [42]:
res_i.dir['templates'] =  os.path.join(res_i.dir['home'],'..','templates','')

In [69]:
res_i.make_dir()

In [70]:
calc_i = streamm.Gaussian('g_methane_HF')

In [71]:
calc_i.strucC = ME

In [72]:
calc_i.set_resource(res_i)

In [73]:
calc_i.make_dir()

In [74]:
os.chdir(calc_i.dir['scratch'])

In [75]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [76]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [77]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

In [78]:
calc_i.properties['commands'] = 'HF/3-21G OPT'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

In [80]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

In [81]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.sh'%(calc_i.tag))

In [82]:
os.chdir(calc_i.dir['home'])
calc_i.dump_json()

In [83]:
os.chdir(calc_i.dir['scratch'])

Save details in .json files 

In [37]:
proj_i.dump_json()

In [38]:
res_i.dump_json()

This sets the current working directory as the root/home directory by default

In [ ]:
pprint(res_i.dir)

Let's create a new home directory called 'example_proj'

In [ ]:
EXAMPLE_DIR = res_i.dir['home']

In [ ]:
new_root = os.path.join(EXAMPLE_DIR,'example_proj')

In [ ]:
res_i.set_home(new_root)

In [ ]:
pprint(res_i.dir)

However, we want to use structures from our previous structures and forcefields examples, so let's set the materials directory to examples/

In [ ]:
res_i.dir['materials'] = EXAMPLE_DIR

To write out input files we will use the templates provided in the streamm package

Set the template dir dictionary entry to the location of templates directory 

In [ ]:
res_i.dir['templates'] =  os.path.join(EXAMPLE_DIR,'..','templates','')

In [ ]:
print res_i.dir['templates']

This also contains the properties dictionary, which can be used to write .pbs scripts on clusters

In [ ]:
pprint(res_i.properties)

By default the resource type is 'local'; however, setting type to 'ssh' will invoke an scp command when copying files

Okay create the directories we need for our calculation

In [ ]:
res_i.make_dir()

Now we should have a directory 'example_proj/' with materials, scratch,   scripts,   storage and   templates directories 

We can create a gaussian calculation

In [ ]:
import streamm.calculations.gaussian as gaussian  

In [ ]:
calc_i = gaussian.Gaussian('methane_HF')

Set the resource and all the directories 

In [ ]:
calc_i.set_resource(res_i)

In [ ]:
pprint(calc_i.dir)

Make the calculation directories

In [ ]:
calc_i.make_dir()

Let's assign a structure to this calculation 

First copy the .xyz file from the materials directory to our scratch directory using the cp_file() function.

This takes an type and key to set the calc_i.files[type][key] dictionary 

In [ ]:
file_type = 'input'
file_key = 'xyz'
file_name = "methane.xyz"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Generally the materials directory is thought to contain a repository of material files, and local versions in the scratch directory should be made in case modifications are necessary 

Change to the scratch directory 

In [ ]:
pprint(calc_i.dir['scratch'])

In [ ]:
os.chdir(calc_i.dir['scratch'])

Read in methane .xyz file from the structures example

In [ ]:
calc_i.strucC.read_xyz('methane.xyz')

In [ ]:
print calc_i.strucC.n_particles

Now that we have a structure and parameters for each interaction we can create an input file for a simulation

Get the bash run script for gaussian. By setting the file_key to run, this will be the script that exicuted when the run() function is called

In [ ]:

file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Get the .com template

In [ ]:

file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Make sure we are in the scratch directory

In [ ]:
pprint(os.getcwd())

Load the template files into memory 

In [ ]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

Set the properties strings in the template files to have the values we want

In [ ]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

Replace the strings in the files['input']['com']

In [ ]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

Add the name of the .com file to the properties, and replace the strings in the files['input']['run']

In [ ]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.sh'%(calc_i.tag))

Save a .json file in the home directory 

In [ ]:
os.chdir(calc_i.dir['home'])
calc_i.dump_json()

Go to scratch directory and see if there is a completed output file for the calculation

In [ ]:
os.chdir(calc_i.dir['scratch'])
calc_i.check()

Check the status 

In [ ]:
pprint("Calculation:{} has status:{}".format(calc_i.tag,calc_i.meta['status']))

If you have gaussian installed on your machine and g09 in your PATH you can run the bash script 

In [ ]:
calc_i.run()

You can read in the data from the log file 

In [ ]:
calc_i.add_file('output','log','{}.log'.format(calc_i.strucC.tag))

In [ ]:
calc_i.check()
if(calc_i.meta['status'] == 'finished' ):
    calc_i.analysis()

Then compress the results and copy them to storage 

In [ ]:
calc_i.store()

Next we can follow a similar procedure to run a LAMMPS MD simulation

In [ ]:
import streamm.calculations.lammps as lammps  

In [ ]:
calc_j = lammps.LAMMPS('methane_lmp')

Set the resource 

In [ ]:
calc_j.set_resource(res_i)

Make directories

In [ ]:
calc_j.make_dir()

In [ ]:
pprint(calc_j.dir)

This takes an type and key to set the calc_i.files[type][key] dictionary 

In [ ]:
file_type = 'input'
file_key = 'xyz'
file_name = "methane.xyz"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
os.chdir(calc_j.dir['scratch'])

Read in methane .xyz file from the structures example

In [ ]:
calc_j.strucC.read_xyz('methane.xyz')

In [ ]:
print calc_j.strucC.n_particles

Set the forcefield particletypes 

In [ ]:
for pkey,p in calc_j.strucC.particles.iteritems():
    if( p.symbol == 'C' ):
        p.ffkey = 'CT'
    elif( p.symbol == 'H' ):
        p.ffkey = 'HC'

Set neighbor list 

In [ ]:
calc_j.strucC.bonded_nblist = calc_j.strucC.guess_nblist(0,radii_buffer=1.25)

Find bonds and bond angles based on neighbor list 

In [ ]:
calc_j.strucC.bonded_bonds()
calc_j.strucC.bonded_angles()

Copy the pickled forcefield parameter file to scratch and read it in

In [ ]:
file_type = 'input'
file_key = 'param'
file_name = "oplsaa.pkl"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
import streamm.forcefields.container as container 

In [ ]:
calc_j.paramC = container.read_pickle('oplsaa')

Use the set_ffparam() function to iterate through the structure container and set parameters based on ffkeys

In [ ]:
calc_j.set_ffparam()

Now we have a structure that has forcefield parameters for each particle,bond and bond angle

Let's get the input file template 

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Bash run file 

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Got to scratch dir

In [ ]:
os.chdir(calc_j.dir['scratch'])

Read in template files

In [ ]:
calc_j.load_str('templates','in')
calc_j.load_str('templates','run')

Write LAMMPS data file

In [ ]:
calc_j.write_data()

Replace properties strings in template and write template 

In [ ]:
calc_j.replacewrite_prop('in','input','in','%s.in'%(calc_j.tag))

Set .in file in properties and write run script 

In [ ]:
calc_j.properties['input_in'] = calc_j.files['input']['in']
calc_j.replacewrite_prop('run','scripts','run','%s.sh'%(calc_j.tag))

Save a .json file in the home directory 

In [ ]:
os.chdir(calc_j.dir['home'])
calc_j.dump_json()

Go to scratch directory and see if there is a completed output file for the calculation

In [ ]:
os.chdir(calc_j.dir['scratch'])
calc_j.check()

In [ ]:
pprint("Calculation:{} has status:{}".format(calc_j.tag,calc_j.meta['status']))

So now we have two calculations, let's put them in a project so we can opperate on them both at the same time

In [ ]:
import streamm.calculations.project as project  

In [ ]:
import copy

In [ ]:
proj_i = project.Project('example_proj')

In [ ]:
proj_i.calculations[calc_i.tag] = copy.deepcopy(calc_i)
proj_i.calculations[calc_j.tag] = copy.deepcopy(calc_j)

Now we can check the status of each calculation with a single command 

In [ ]:
proj_i.check()

We can run each simulation

In [ ]:
proj_i.run()

And, we can tar up the results and copy the tar files to a storage location

In [ ]:
proj_i.store()

Neat!